In [1]:
import spacy

spacy.require_gpu()

/usr/local/lib/python3.8/dist-packages/cupy/_environment.py:435: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda110, cupy-cuda113

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


True

In [2]:
!python -m spacy train config.cfg --verbose --output ner_demo/training/  --paths.train train.spacy --paths.dev valid.spacy  --gpu-id 0

/usr/local/lib/python3.8/dist-packages/cupy/_environment.py:435: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda110, cupy-cuda113

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
2022-03-10 15:03:57.048454: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'l

In [18]:
import pandas as pd
import spacy
test_abstract_df =pd.read_csv('../part2/data/abstracts_test.csv', sep='\t')
test_abstract_df.head()

,abstract_id,title,abstract
0,1711760,Delayed institution of hypertension during foc...,The effect of induced hypertension instituted ...
1,6086495,Localisation of the Becker muscular dystrophy ...,A linkage study in 30 Becker muscular dystroph...
2,7018927,Pituitary response to luteinizing hormone-rele...,The effects of a 6-hour infusion with haloperi...
3,7811247,X-linked adrenoleukodystrophy (ALD): a novel m...,Fragments of the adrenoleukodystrophy (ALD) cD...
4,8944024,Detection of heterozygous mutations in BRCA1 u...,The ability to scan a large gene rapidly and a...


In [20]:
ner = spacy.load(R"ner_demo/training/model-best") #load the best model

In [21]:
# display sentence involving original entities
test_sample =test_abstract_df.iloc[1]['abstract']
spacy.displacy.render(ner(test_sample),jupyter=True, style = "ent")


In [23]:


rec_entity_mentions= []
#test_sentences = [x[0] for x in TEST_DATA[0:4000]] # extract the sentences from [sentence, entity]
for i, row in test_abstract_df.iterrows():
    abs_id = row['abstract_id']
    text = row['title']+' '+row['abstract']
    doc = ner(text)
    for ent in doc.ents:
        #print(ent.text, ent.start_char, ent.end_char, ent.label_)
        rec_entity_mentions.append([i, abs_id, ent.start_char, ent.end_char,  ent.label_])
    #displacy.render(doc,jupyter=True, style = "ent")

In [24]:
submission_df = pd.DataFrame(rec_entity_mentions, columns=['id','abstract_id','offset_start','offset_finish','type'])

In [26]:
submission_df.to_csv('submission_10.csv', sep='\t', index=False)

In [27]:
!tar -zcvf ner_demo.tar.gz ner_demo

ner_demo/
ner_demo/training/
ner_demo/training/model-best/
ner_demo/training/model-best/tokenizer
ner_demo/training/model-best/transformer/
ner_demo/training/model-best/transformer/model
ner_demo/training/model-best/transformer/cfg
ner_demo/training/model-best/ner/
ner_demo/training/model-best/ner/moves
ner_demo/training/model-best/ner/model
ner_demo/training/model-best/ner/cfg
ner_demo/training/model-best/vocab/
ner_demo/training/model-best/vocab/vectors.cfg
ner_demo/training/model-best/vocab/strings.json
ner_demo/training/model-best/vocab/vectors
ner_demo/training/model-best/vocab/lookups.bin
ner_demo/training/model-best/vocab/key2row
ner_demo/training/model-best/meta.json
ner_demo/training/model-best/config.cfg
ner_demo/training/model-last/
ner_demo/training/model-last/tokenizer
ner_demo/training/model-last/transformer/
ner_demo/training/model-last/transformer/model
ner_demo/training/model-last/transformer/cfg
ner_demo/training/model-last/ner/
ner_demo/training/model-last/ner/moves


In [29]:
entities_df = pd.read_csv('../part2/data/entities_test.csv', sep='\t')
entities_df.head()

,id,abstract_id,offset_start,offset_finish,type,mention,entity_ids
0,0,1711760,23,35,DiseaseOrPhenotypicFeature,hypertension,D006973
1,1,1711760,49,66,DiseaseOrPhenotypicFeature,cerebral ischemia,D002545
2,2,1711760,78,89,DiseaseOrPhenotypicFeature,brain edema,D001929
3,3,1711760,113,125,DiseaseOrPhenotypicFeature,hypertension,D006973
4,4,1711760,165,197,DiseaseOrPhenotypicFeature,middle cerebral artery occlusion,D020244


In [30]:
id2abstr ={ abs_id: abs_text for abs_id, abs_text in test_abstract_df[['abstract_id','abstract']].values }

id2title ={ abs_id: title for abs_id, title in test_abstract_df[['abstract_id','title']].values }

In [31]:
testing_data =[]

for key, df in entities_df.groupby('abstract_id'):
    #print (key)
    entity_mentions = []
    text = id2title[key]+' '+id2abstr[key]
    text = text.lower()
    for i, row in df.iterrows():
          entity_mention = (row['offset_start'], row['offset_finish'],row['type'])
          #print (text[row['offset_start']:row['offset_finish']])
          entity_mentions.append(entity_mention)
    testing_data.append( (text,  {'entities': entity_mentions}))

In [32]:
#import pandas as pd
from tqdm import tqdm
#import spacy
from spacy.tokens import DocBin
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object
for text, annot in tqdm(testing_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
            print (start, end, label)
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("test.spacy") # save the docbin object


/usr/local/lib/python3.8/dist-packages/spacy/util.py:1635: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
 53%|█████▎    | 53/100 [00:00<00:00, 244.96it/s]

Skipping entity
1285 1289 GeneOrGeneProduct
Skipping entity
607 621 ChemicalEntity
Skipping entity
95 101 SequenceVariant
Skipping entity
583 589 SequenceVariant
Skipping entity
1167 1173 SequenceVariant
Skipping entity
1227 1233 SequenceVariant
Skipping entity
1242 1248 SequenceVariant
Skipping entity
1494 1500 SequenceVariant
Skipping entity
1562 1568 SequenceVariant
Skipping entity
1754 1760 SequenceVariant
Skipping entity
1878 1884 SequenceVariant
Skipping entity
1327 1346 DiseaseOrPhenotypicFeature
Skipping entity
582 590 ChemicalEntity
Skipping entity
593 604 ChemicalEntity
Skipping entity
40 49 SequenceVariant
Skipping entity
345 355 ChemicalEntity
Skipping entity
357 393 ChemicalEntity
Skipping entity
650 654 GeneOrGeneProduct
Skipping entity
694 698 GeneOrGeneProduct
Skipping entity
796 800 GeneOrGeneProduct
Skipping entity
819 823 GeneOrGeneProduct
Skipping entity
1064 1068 GeneOrGeneProduct
Skipping entity
1142 1146 GeneOrGeneProduct
Skipping entity
1190 1194 GeneOrGeneProdu

100%|██████████| 100/100 [00:00<00:00, 266.90it/s]


 640 GeneOrGeneProduct
Skipping entity
646 648 DiseaseOrPhenotypicFeature
Skipping entity
650 652 GeneOrGeneProduct
Skipping entity
872 874 GeneOrGeneProduct
Skipping entity
1128 1130 GeneOrGeneProduct
Skipping entity
1024 1032 GeneOrGeneProduct
Skipping entity
1409 1412 GeneOrGeneProduct
Skipping entity
1415 1419 GeneOrGeneProduct
Skipping entity
1422 1427 GeneOrGeneProduct
Skipping entity
1099 1104 GeneOrGeneProduct
Skipping entity
1106 1110 GeneOrGeneProduct
Skipping entity
1714 1718 GeneOrGeneProduct
Skipping entity
1720 1744 GeneOrGeneProduct
Skipping entity
210 213 GeneOrGeneProduct
Skipping entity
215 219 GeneOrGeneProduct
Skipping entity
1170 1173 GeneOrGeneProduct
Skipping entity
1176 1181 GeneOrGeneProduct
Skipping entity
1184 1189 GeneOrGeneProduct
Skipping entity
1081 1086 GeneOrGeneProduct
Skipping entity
1088 1095 GeneOrGeneProduct
Skipping entity
1343 1346 GeneOrGeneProduct
Skipping entity
120 124 GeneOrGeneProduct
Skipping entity
124 128 SequenceVariant
Skipping entity


In [33]:
!spacy evaluate ner_demo/training/model-best test.spacy --gpu-id 0

/usr/local/lib/python3.8/dist-packages/cupy/_environment.py:435: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda110, cupy-cuda113

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''
2022-03-10 16:40:43.531060: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'l